In [11]:
import pandas as pd
import folium
plt.style.use('bmh')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 100)

In [7]:
f = pd.read_csv('bayut_0.5_outliers.csv')
f.replace('Jumeirah Heights', 'Jumeirah Islands',regex=True, inplace=True)

In [8]:
f['county'] = f['county'].str[1:]
f['county'] = f['county'].str.split('(')
f['county'] = [i[0] for i in f.county]


In [9]:
from altair import Chart, load_dataset
import altair as alt
# load built-in dataset as a pandas DataFrame
coords = pd.read_csv('coords.csv')
m = folium.Map(location=[25.2048, 55.2708])
for i in range(len(coords)):
    c = coords.iloc[i]['county'][1:]
    lo = coords.iloc[i]['long']
    la = coords.iloc[i]['lat']
    cars = f[f['county']== c][['area','beds','price','baths']]
    cars.area = np.ceil(cars.area)
    cars = cars.replace(0.6, 0)
    
    try:   
        beds_slider = alt.binding_range(min=min(cars.beds.values), max=max(cars.beds.values), step=1)
        slider_selection = alt.selection_single(bind=beds_slider, fields=['beds'],name='no. of ', init={'beds': 2})

        baths_slider2 = alt.binding_range(min=min(cars.baths.values), max=max(cars.baths.values), step=1)
        slider_selection2 = alt.selection_single(bind=baths_slider2, fields=['baths'],name='no. of', init={'baths': 2})

        chart = alt.Chart(cars).mark_bar().encode(
            alt.X('area:Q',bin=True),
            y='average(price)',
        #     color='beds:N',
        ).add_selection(
            slider_selection
        ).transform_filter(
            slider_selection
        ).add_selection(
            slider_selection2
        ).transform_filter(
            slider_selection2
        )

        vega = folium.features.VegaLite(
            chart,
            width='100%',
            height='100%',
        )

        marker = folium.features.Marker([la, lo], tooltip=c)

        popup = folium.Popup()

        vega.add_to(popup)
        popup.add_to(marker)

        marker.add_to(m)
    except:
        continue

In [10]:
m

In [12]:
m.save('final_map.html')